# P3 - Open Street Map Project

##### Create sample the OSM file

OSM file is much too bit to work with - make a sample viable for development.

In [30]:
import xml.etree.ElementTree as ET  # Use cElementTree or lxml if too slow

OSM_FILE = "/Users/mchana/GitHub/udacity/large_files/new-orleans-city.osm"  # Replace this with your osm file
SAMPLE_FILE = "supporting_files/new-orleans-city_sample_k1000_relations.osm"

k = 1000 # Parameter: take every k-th top level element

# def get_element(osm_file, tags=('node', 'way', 'relation')):
# def get_element(osm_file, tags=('way',)):
def get_element(osm_file, tags=('relation',)):
    """Yield element if it is the right type of tag

    Reference:
    http://stackoverflow.com/questions/3095434/inserting-newlines-in-xml-file-generated-via-xml-etree-elementtree-in-python
    """
#     context = iter(ET.iterparse(osm_file, events=('start', 'end')))
    context = ET.iterparse(osm_file, events=('start', 'end'))
    _, root = next(context)
    for event, elem in context:
        if event == 'end' and elem.tag in tags:
            yield elem
        root.clear()


with open(SAMPLE_FILE, 'w') as output:
    output.write("""<?xml version="1.0" encoding="UTF-8"?>\n""")
    output.write('<osm>\n  ')

    # Write every kth top level element
    for i, element in enumerate(get_element(OSM_FILE)):
        if i % k == 0:
            output.write(ET.tostring(element, encoding='unicode'))

    output.write('</osm>')

##### Manually inspect the data

From a visual perspective, how is the data structured?

How is the sample osm file structured?

* osm element is the root
* Only sub-elements I'm interested in are 'node' (a single lat/lon point, possibly with 'tags'), 'way'(roads, boundaries, with nested 'nodes ' and also possibly with 'tags'), 'relation'(logical or geographic relationship between elements, with nested 'member', 'tags', etc. - ex. 'ways' making up a "route" for cycling or hiking)
* Each 'node' nested in a 'way' can be defined elsewhere, but is referenced in the 'way'
* main things that seem to be present (as ways) are highways, natural (water bodies, etc.) and buildings
* Appears 'nodes' that are nested in 'ways' are first defined at beginning of XML document, 'ways' are in the middle, and 'relations' are at the end
* Appears a node can be:
    * basically a lat and lon 
    * be a lat and lon but with nested 'tags' with other info, like name of a church, type of religion, street address, etc.
    
    
Questions:
* Why do some 'node's have the street address for a building? Are they supposed to be there (ex. a 'way' representing a building will reference that 'node' to point to the address) or is this a human mistake?

I should parse the data and get it into some structure (python list, dict, etc. or even pandas df) so that I can test fields for Validity, Accuracy, Completeness, Consistency and Uniformity.